# Hybrid Demand Forecasting and ROP Planning for Alton 20

In [4]:
import pandas as pd
import numpy as np
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt



In [9]:
import pandas as pd

# Define the file path
file_path = "/Users/ashikurrahmanshihab/GitHub/Hybrid-Demand-Forecasting-and-Reorder-Point-Planning-Using-AROPHET-in-the-Pharmaceutical-Industry/data.xlsx"

# Read the Excel file (adjust sheet name if needed)
sales_data = pd.read_excel(file_path)

# Show the first 5 rows
sales_data.head()


,ds,y,marketing_spend,doctor_visits,disease_trend,stock_level,lead_time_days,safety_stock,shelf_life_months,MOQ
0,2014-01-01,1187.000000,892,5048,346,6889,14,500,36,3000
1,2014-02-01,1332.860756,1555,4613,395,6701,7,1000,12,3000
2,2014-03-01,1342.644005,1301,5012,230,6291,14,1000,12,2000
3,2014-04-01,1407.575042,1755,5248,328,5841,14,500,36,1000
4,2014-05-01,1386.842733,1262,5725,267,7576,7,500,12,1000


In [10]:

# Set up Prophet model with regressors
m = Prophet()
m.add_regressor('marketing_spend')
m.add_regressor('doctor_visits')
m.add_regressor('disease_trend')
m.add_regressor('stock_level')

# Fit model
m.fit(sales_data.rename(columns={"ds": "ds", "y": "y"}))


02:18:58 - cmdstanpy - INFO - Chain [1] start processing
02:18:58 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:

future = m.make_future_dataframe(periods=12, freq='MS')
future = pd.merge(future, sales_data, on='ds', how='left')

# Fill future regressor values (naive method: forward fill)
for col in ['marketing_spend', 'doctor_visits', 'disease_trend', 'stock_level']:
    future[col] = future[col].ffill()

forecast = m.predict(future)
m.plot(forecast)
plt.show()


In [ ]:

# Calculate residuals
sales_data['forecast'] = m.predict(sales_data)['yhat']
sales_data['residuals'] = sales_data['y'] - sales_data['forecast']

# Fit ARIMA on residuals
model_arima = ARIMA(sales_data['residuals'], order=(1,0,1))
model_arima_fit = model_arima.fit()

# Forecast residuals
arima_forecast = model_arima_fit.forecast(steps=12)


In [ ]:

# Combine Prophet + ARIMA (hybrid)
final_forecast = forecast.tail(12).copy()
final_forecast['yhat_hybrid'] = final_forecast['yhat'] + arima_forecast.values

# Plot
plt.figure(figsize=(10,6))
plt.plot(sales_data['ds'], sales_data['y'], label='Actual')
plt.plot(final_forecast['ds'], final_forecast['yhat_hybrid'], label='Hybrid Forecast')
plt.legend()
plt.show()


In [ ]:

# Simple ROP = demand during lead time + safety stock
avg_lead_time = sales_data['lead_time_days'].mean()
daily_demand = sales_data['y'].mean() / 30  # crude estimate

ROP = avg_lead_time * daily_demand + sales_data['safety_stock'].mean()
print(f"Recommended Reorder Point (ROP): {ROP:.2f} units")
